## Set the environment

In [1]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import flux_variability_analysis
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import escher
from escher import Builder
from utils import show_map
from utils.check_precursor_problem import check_precursor_problem
map_loc = "../data/overallmap.json" #Import the map that has been drawn using escher.

## Read the model and the transcriptomics

### Read the model

In [2]:
model = cobra.io.load_json_model('../models/updated_model.json')
model.reactions.NGAM.bounds = (0,1000)
model.objective = model.reactions.NGAM

In [3]:
reaction = Reaction('aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN')
reaction.name = 'aa_ser_gly_thr_4.3.1.19_L_SERINE_AMMONIA_LYASE_RXN'
reaction.lower_bound = 0.  # This is the default
reaction.upper_bound = 1000.  # This is the default
model.add_reactions([reaction])
reaction.add_metabolites({"L__serine":-1.0,
                         "pyruvate":1.0,
                         "ammonia": 1.0})
model.reactions.get_by_id('aa_met_salvage_4.2.1.22_CYSTATHIONINE__BETA__SYNTHASE__RXN').bounds = (0,0)# these two reactions do not exist in this organism

### Read the transcriptomics

In [4]:
df_gene_reaction_rule =pd.read_csv('../data/reaction_id_RAS.csv')
# set the reaction id column as the index in order to map it to the model
df_gene_reaction_rule = df_gene_reaction_rule.fillna(0)# This code can be used to set the Nan values to 0. 
df_gene_reaction_rule = df_gene_reaction_rule.set_index(["reaction_id"],)# This code can be used to set the index.
df_gene_reaction_rule.head()

,Unnamed: 0,Reaction name,pathway,genes,gene_reaction_rule,Unnamed: 5,Unnamed: 6,Array_67 70,Array_67 80,Array_68 70,Array_68 80,Array_86 70,Array_86 80,Array_85 70,Array_85 80,average_list_column_70,average_list_column_80
reaction_id,,,,,,,,,,,,,,,,,
T_ABC__transporter_beta__D__glucose,0,ABC transport,0,"28,473,066,284,830,600,000,000,000,000,000",and,0.0,0,142229.0,63180.0,0.0,0.0,196101.0,99840.0,215040.0,141324.0,184456.6667,101448.0
carb_entner_1.1.1.359_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN,1,GDH,0,"300,330,423,204",or,0.0,0,105265.0,111072.0,134460.0,107596.0,134495.0,154112.0,150784.0,92612.0,131251.0000,116348.0
carb_entner_1.1.1.47_GLUCOSE__1__DEHYDROGENASE__NAD+__RXN,2,GDH,0,"300,330,423,204",or,0.0,0,105265.0,111072.0,134460.0,107596.0,134495.0,154112.0,150784.0,92612.0,131251.0000,116348.0
carb_entner_1.1.1.360_GLUCOSE__1__DEHYDROGENASE__NADP+__RXN,3,GDH,0,"300,330,423,204",or,0.0,0,105265.0,111072.0,134460.0,107596.0,134495.0,154112.0,150784.0,92612.0,131251.0000,116348.0
carb_entner_3.1.1.17_GLUCONOLACT__RXN,4,GL,0,spontaneous,0,0.0,0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0,1000000.0000,1000000.0


## Write a function for mapping transcriptomics to model reaction bounds

In [5]:
def map_the_RNA_to_model (a, b,c):# Here a is the number used to divid the transcriptome data, b is the column for the transcriptome data, -c is the lower bound for glucose exchange reaction 
    
    model1 = model.copy()
    model1.reactions.get_by_id('T_flux_beta__D__glucose').bounds = (-c,0)
    model1.reactions.get_by_id("aa_ser_gly_cys_sec_trp_1.1.1.95_PGLYCDEHYDROG__RXN").bounds = (0,0)

    for rxn in df_gene_reaction_rule.index:

        the_bound = a*df_gene_reaction_rule[b].loc[rxn]

        if model1.reactions.get_by_id(rxn).lower_bound < 0 and model1.reactions.get_by_id(rxn).upper_bound > 0: # this finds reversible rxns

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, the_bound) # we set the bound(s) with max abs(FVA) 

        elif model1.reactions.get_by_id(rxn).upper_bound > 0: # irreversible, flows in forward direction

            model1.reactions.get_by_id(rxn).bounds = (0, the_bound)  # the lower bound will stay zero

        elif int(model1.reactions.get_by_id(rxn).lower_bound) < 0: # irreversible, flows in backward direction

            model1.reactions.get_by_id(rxn).bounds = (-1*the_bound, 0) # the upper bound will stay zero
    model1.objective  = model1.reactions.NGAM
    sol = model1.optimize()
    b = show_map(sol,map_loc)
    return(b)

## Map the transcriptomics at different temperatures 70 and 80 ℃

### 70℃

#### The effect of the glucose concentrations with fixed a
1. a is fixed at 10-5, and bound of glucose exchange reaction is changed. 
2. When the bound of glucose exchange reaction lower than 0.745, the cell will only use pgk and gapdh pathway.
3. when the bound of glucose exchange reaction higher than 0.745, the cell will use pgk and gapdh pathway and non-phosphorylation pathway.
4. The reason for this is because that PGM reaction becomes the control reaction, since the RNA level of this enzyme is realatively low. 

##### Figure S5

In [6]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [7]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',2)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [22]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',0.8)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [21]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',0.745)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [8]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',0.5)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

#### The effect of the a with a fixed glucose concentrations 
1. c is fixed at 1, and the a is changed. 
2. when a is smaller than 1/74500, the cell will use pgk and gapdh pathway and non-phosphorylation pathway.
2. when a is larger than 1/74500, the cell will use pgk and gapdh pathway.
4. The reason for this is because that PGM reaction becomes the control reaction, since the RNA level of this enzyme is realatively low. 

In [9]:
map_the_RNA_to_model (0.5*10 ** -5,'average_list_column_70',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [10]:
map_the_RNA_to_model (10 ** -5,'average_list_column_70',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [11]:
map_the_RNA_to_model (1/74500,'average_list_column_70',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [12]:
map_the_RNA_to_model (1/50000,'average_list_column_70',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [13]:
map_the_RNA_to_model (1/100000,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

### 80℃

#### The effect of the glucose concentrations with fixed a
with the fixed a, the cell will always use gapdh and pgk pathway. 

##### Figure S6

In [14]:
map_the_RNA_to_model (10 ** -5,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [15]:
map_the_RNA_to_model (10 ** -5,'average_list_column_80',0.5)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [16]:
map_the_RNA_to_model (10 ** -5,'average_list_column_80',1.5)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

#### The effect of the a with  the fixed glucose concentrations
with the fixed glucose concentration, the cell will always use gapdh and pgk pathway. 

In [17]:
map_the_RNA_to_model (1/200000,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [18]:
map_the_RNA_to_model (10 ** -5,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [19]:
map_the_RNA_to_model (1/74500,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…

In [20]:
map_the_RNA_to_model (1/50000,'average_list_column_80',1)

Builder(hide_secondary_metabolites=False, highlight_missing=True, reaction_data={'Biomass': 0.0, 'Biomass__fuc…